## 03. Zustand로 상태 관리

<br>

### Zustand 설치

```bash
$ npm install zustand
```

<br>

### Zustand 스토어 생성
- **`create<StoreType>`** : 스토어에 포함될 상태와 액션의 타입을 제네릭으로 지정
- **`(set, get) => ({...})`** : 상태와 액션을 정의하는 함수
  - **`set`**은 Zustand에서 상태를 변경할 때 사용 
  - **`get`**은 현재 상태를 조회할 때 사용하며 생략 가능
- **`set`** : 상태를 변경할 때 사용하는 함수

```ts
create<StoreType>()((set, get) => ({
    state: 초깃값,
    action: () => set(...)
}))
```

<br>

- `src/store/counterStore.ts`

```ts
import { create } from "zustand";

interface CounterStoreState {
  count: number;
  increment: () => void;
  decrement: () => void;
  reset: () => void;
  resetIfEven: () => void;
}

export const useCounterStore = create<CounterStoreState>((set, get) => ({
  count: 0,
  increment: () => set((state) => ({ count: state.count + 1 })), // 기존 상태 기반
  decrement: () => set((state) => ({ count: state.count - 1 })), // 기존 상태 기반
  reset: () => set({ count: 0 }), // 새 상태 직접 지정
  resetIfEven: () => {
    const { count } = get();
    if (count % 2 === 0) {
      set({ count: 0 });
    }
  },
}));
```

<br>

### Zustand 스토어 사용
- `App.tsx`

```ts
import Count from "./components/Count";
import CountOutsideDisplay from "./components/CountOutsideDisplay";

export default function App() {
  return (
    <>
      <Count />
      <CountOutsideDisplay />
    </>
  );
}
```

<br>

- `src/components/Count.tsx`

```ts
import CountButtons from "./CountButtons";
import CountDisplay from "./CountDisplay";

export default function Count() {
  return (
    <>
      <CountDisplay />
      <CountButtons />
    </>
  );
}
```

<br>

- `src/components/CountButtons.tsx`

```ts
import { useCounterStore } from "../store/counterStore";

export default function CountButtons() {
  const increment = useCounterStore((state) => state.increment);
  const decrement = useCounterStore((state) => state.decrement);
  const reset = useCounterStore((state) => state.reset);
  const resetIfEven = useCounterStore((state) => state.resetIfEven);
  return (
    <>
      <button onClick={decrement}>감소</button>
      <button onClick={reset}>초기화</button>
      <button onClick={resetIfEven}>초기화(짝수)</button>
      <button onClick={increment}>증가</button>
    </>
  );
}
```

<br>

* `src/components/CountDisplay.tsx`

```ts
import { useCounterStore } from "../store/counterStore";

export default function CountDisplay() {
  const count = useCounterStore((state) => state.count);
  return <h1>Count: {count}</h1>;
}
```

<br>

- `src/components/CountOutsideDisplay.tsx`

```ts
import { useCounterStore } from "../store/counterStore";

export default function CountOutsideDisplay() {
  const count = useCounterStore((state) => state.count);
  return <h1>Outside Count: {count}</h1>;
}
```

<br>

### Zustand의 고급 기능
- **Zustand는 미들웨어를 제공해 상태 관리 기능을 확장할 수 있도록 지원**
  - 미들웨어 : 상태를 읽거나 쓸 때 또는 변경하는 과정에 추가 동작을 삽입할 수 있게 해주는 기능
- 주요 미들웨어 : `persist`, `subscribeWithSelector`, `immer`, `devtools` 등

<br>

#### `persist`
- **상태를 로컬 스토리지에 저장할 수 있어 페이지를 새로 고쳐도 변경한 상태가 초기화되지 않고 유지**
- `src/store/countStore.ts`
  - `persist()`
    - **첫 번째 인자 : 상태와 액션을 정의하는 함수**
    - **두 번째 인자 : 상태를 어디에, 어떻게 저장할지 설정하는 객체**
      - `name`은 로컬 스토리지에 저장할 때 사용할 키 이름


```ts
import { create } from "zustand";
import { persist } from "zustand/middleware";

interface CounterStoreState {

    ...

}

export const useCounterStore = create<CounterStoreState>() (
  persist(
    (set, get) => ({
      count: 0,
      increment: () => set((state) => ({ count: state.count + 1 })),
      decrement: () => set((state) => ({ count: state.count - 1 })),
      reset: () => set({ count: 0 }), 
      resetIfEven: () => {
        const { count } = get();
        if (count % 2 === 0) {
          set({ count: 0 });
        }
      },
    }),
    {
      name: "counter-storage", // 로컬 스토리지에 저장될 키 이름
    }
  )
);
```

<br>

#### `subscribeWithSelector` 
- Zustand는 상태가 변경될 때 특정 동작을 실행할 수 있도록 구독 기능 (`subscribe()`)을 제공
  
  $\rightarrow$ **특정 상태를 감시하다가, 변경이 되는 순간 자동으로 지정한 함수가 실행되도록 설정**
- **`subscribe()`는 상태 변화를 정밀하게 감지하고 변화된 상태를 저장하기 위해 `subscribeWithSelector`, `persist` 미들웨어와 조합하여 함께 사용**
  - **첫 번째 인자 : 어떤 상태를 구독할지 결정하는 선택자 함수**
  - **두 번째 인자 : 해당 상태가 변경될 때 실행할 함수**

<br>

- `src/store/useCounterStore.ts`

```ts
import { create } from "zustand";
import { persist, subscribeWithSelector } from "zustand/middleware";

interface CounterStoreState {
    
    ...

}

export const useCounterStore = create<CounterStoreState>() (
  subscribeWithSelector(
    persist(
      (set, get) => ({
        count: 0,
        increment: () => set((state) => ({ count: state.count + 1 })), // 기존 상태 기반
        decrement: () => set((state) => ({ count: state.count - 1 })), // 기존 상태 기반
        reset: () => set({ count: 0 }), // 새 상태 직접 지정
        resetIfEven: () => {
          const { count } = get();
          if (count % 2 === 0) {
            set({ count: 0 });
          }
        },
      }),
      {
        name: "counter-storage", // 로컬 스토리지에 저장될 키 이름
      }
    )
  )
);
```

<br>

- `src/components/CountButtons.tsx`
  - `Count` 상태를 구독하며 갑 변경 감지

```ts
import { useEffect } from "react";
import { useCounterStore } from "../store/counterStore";

export default function CountButtons() {
  const increment = useCounterStore((state) => state.increment);
  const decrement = useCounterStore((state) => state.decrement);
  const reset = useCounterStore((state) => state.reset);
  const resetIfEven = useCounterStore((state) => state.resetIfEven);

  useEffect(() => {
    const unsubscribe = useCounterStore.subscribe(
      // 상태 구독 설정
      (state) => state.count, // 구독할 상태 선택
      (newCount) => {
        // 상태가 변경될 때 실행할 함수
        console.log("Count has changed to:", newCount);
      }
    );
    return () => {
      // 컴포넌트 언마운트 시 구독 해제
      unsubscribe();
    };
  }, []);

  return (
    
    ...

  );
}
```

<br>

#### `immer`
- **Zustand에서 상태를 변경할 때는 반드시 불변성을 지켜야 함**
  
  **즉, 기존 상태를 직접 수정하지 않고 새로운 객체로 상태를 업데이트 해야함**

  $\rightarrow$ **상태가 복잡해질수록 코드는 길고 복잡해지며, 가독성이 떨어지고 실수하기 쉬움**

  $\rightarrow$ **`immer`를 사용하여, 상태를 직접 수정하듯 간단하게 처리**

<br>

- **설치**

```bash
$ npm install immer
```

<br>

- `src/components/CountButtons.tsx`

```ts
import { create } from "zustand";
import { persist, subscribeWithSelector } from "zustand/middleware";
import { immer }  from "zustand/middleware/immer";

interface CounterStoreState {

    ...

}

export const useCounterStore = create<CounterStoreState>() (
  subscribeWithSelector(
    persist(
      immer(
        (set, get) => ({
          count: 0,
          // immer가 자동으로 불변성 관리
          increment: () => set((state) => { state.count += 1 }),
          decrement: () => set((state) => { state.count -= 1 }),
          reset: () => set((state) => { state.count = 0}),
          resetIfEven: () => {
            const { count } = get();
            if (count % 2 === 0) {
              set(state => state.count = 0);
            }
          },
        }),
      ),
      {
        name: "counter-storage", // 로컬 스토리지에 저장될 키 이름
      }
    )
  )
);
```

<br>

#### `devtools`
- **Redux 개발자 도구와 연동**

```ts
import { create } from "zustand";
import { devtools, persist, subscribeWithSelector } from "zustand/middleware";
import { immer }  from "zustand/middleware/immer";

interface CounterStoreState {

    ...

}

export const useCounterStore = create<CounterStoreState>() (
  devtools(
    subscribeWithSelector(
      persist(
        immer(
          (set, get) => ({
            count: 0,
            // immer가 자동으로 불변성 관리
            increment: () => set((state) => { state.count += 1 }),
            decrement: () => set((state) => { state.count -= 1 }),
            reset: () => set((state) => { state.count = 0}),
            resetIfEven: () => {
              const { count } = get();
              if (count % 2 === 0) {
                set(state => state.count = 0);
              }
            },
          }),
        ),
        {
          name: "counter-storage", // 로컬 스토리지에 저장될 키 이름
        }
      )
    ),
    {
      trace: true // 액션 호출 스택 추적 활성화
    }
  )
);
```

<br>

<hr>

<br>

## 05. 할 일 관리 애플리케이션 개선

<br>

### Zustand 스토어 생성

- `src/store/todoStore.ts`

```ts
import { create } from "zustand";
import { persist } from "zustand/middleware";
import { immer } from "zustand/middleware/immer";

interface TodoStoreState {
  todos: Todo[];
}

interface TodoStoreActions {
  addTodo: (title: string) => void;
  deleteTodo: (id: number) => void;
  toggleTodo: (id: number) => void;
  modifyTodo: (id: number, title: string) => void;
}

export const useTodoStore = create<TodoStoreState & TodoStoreActions>()(
  persist(
    immer((set) => ({
      todos: [],
      addTodo: (title) =>
        set((state) => {
          state.todos.push({
            id: new Date().getTime(),
            title,
            done: false,
          });
        }),

      deleteTodo: (id) =>
        set((state) => {
          state.todos = state.todos.filter((todo: Todo) => todo.id !== id);
        }),

      toggleTodo: (id) =>
        set((state) => {
          state.todos = state.todos.map((todo: Todo) =>
            todo.id === id ? { ...todo, done: !todo.done } : todo
          );
        }),
        
      modifyTodo: (id, title) =>
        set((state) => {
          state.todos = state.todos.map((todo: Todo) =>
            todo.id === id ? { ...todo, title } : todo
          );
        }),
    })),
    { name: "todos" }
  )
);
```

<br>

### 스토어 적용
- `props` 전달 및 메모이제이션 코드 제거

<br>

- `App.tsx`
```ts
import TodoEditor from "./components/TodoEditor";
import TodoHeader from "./components/TodoHeader";
import TodoList from "./components/TodoList";

export default function App() {
  return (
    <div className="todo">
      <TodoHeader />
      <TodoEditor />
      <TodoList />
    </div>
  );
}
```

<br>

- `src/components/TodoEditor.tsx`

```ts
import { useState } from "react";
import Button from "./html/Button";
import Input from "./html/Input";
import { useTodoStore } from "../store/todoStore";

export default function TodoEditor() {
  const addTodo = useTodoStore((state) => state.addTodo); // 액션 생성 함수 가져오기
  const [text, setText] = useState("");
  const handleSubmit = (e: React.FormEvent<HTMLFormElement>) => {
    e.preventDefault(); // 기본 동작 막기
    if (!text.trim()) return; // 입력창이 비어 있으면 함수 종료
    addTodo(text); // 부모 컴포넌트에서 전달받은 함수 실행
    setText(""); // 입력 필드 초기화
  };

  return (
    <form className="todo__form" onSubmit={handleSubmit}>
      <div className="todo__editor">
        <Input
          type="text"
          className="todo__input"
          placeholder="Enter Todo List"
          value={text}
          onChange={(e) => setText(e.target.value)}
        />
        <Button className="todo__button" type="submit">
          Add
        </Button>
      </div>
    </form>
  );
}
```

<br>

- `src/components/TodoList.tsx`

```ts
import { useTodoStore } from "../store/todoStore";
import TodoListItem from "./TodoListItem";
import TodoListItemEmpty from "./TodoListItemEmpty";

export default function TodoList() {
  const todos = useTodoStore((state) => state.todos);
  return (
    <ul className="todo__list">
      {todos.length === 0 && <TodoListItemEmpty />}
      {todos.length > 0 &&
        todos.map((todo) => <TodoListItem key={todo.id} todo={todo} />)}
    </ul>
  );
}
```

<br>

- `src/components/TodoListItem.tsx`

```ts
import { memo, useState } from "react";
import Button from "./html/Button";
import Checkbox from "./html/Checkbox";
import SvgClose from "./svg/SvgClose";
import SvgPencil from "./svg/SvgPencil";
import Input from "./html/Input";
import { useTodoStore } from "../store/todoStore";

export default memo(function TodoListItem({ todo }: { todo: Todo }) {
  console.log("TodoListItem renering");
  const deleteTodo = useTodoStore((state) => state.deleteTodo);
  const toggleTodo = useTodoStore((state) => state.toggleTodo);
  const modifyTodo = useTodoStore((state) => state.modifyTodo);

  const [isModify, setIsModify] = useState(false); // 수정 여부를 판단하는 상태 정의
  const [modifyTitle, setModifyTitle] = useState(""); // 수정할 내용을 담는 상태 정의
  
  const modifyHandler = () => {
    setIsModify((modify) => !modify);
    setModifyTitle(modifyTitle === "" ? todo.title : modifyTitle);
    if (modifyTitle.trim() !== "" && modifyTitle !== todo.title) {
      modifyTodo(todo.id, modifyTitle);
    }
  };

  return (
    
    ...

  );
});
```

<br>

<hr>